In [1]:
pip install faker

  Obtaining dependency information for faker from https://files.pythonhosted.org/packages/c0/c3/0451555e7a9a233bc17f128cff7654ec60036d4ccbb8397dd949f28df176/Faker-33.0.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------  1.9/1.9 MB 24.0 MB/s eta 0:00:01
   ---------------------------------------  1.9/1.9 MB 24.0 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 12.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sqlite3
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

conn = sqlite3.connect("library_management_system.db")
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS Authors (
    author_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    birth_year INTEGER NOT NULL,
    nationality TEXT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author_id INTEGER,
    publication_year INTEGER,
    genre TEXT,
    price REAL,
    availability_status TEXT CHECK (availability_status IN ('available', 'checked out', 'reserved')),
    FOREIGN KEY (author_id) REFERENCES Authors(author_id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Members (
    member_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    membership_type TEXT CHECK (membership_type IN ('Basic', 'Premium')),
    joining_date DATE,
    total_books_borrowed INTEGER
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Loans (
    loan_id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER,
    member_id INTEGER,
    borrow_date DATE,
    return_date DATE,
    loan_status TEXT CHECK (loan_status IN ('returned', 'overdue')),
    FOREIGN KEY (book_id) REFERENCES Books(book_id),
    FOREIGN KEY (member_id) REFERENCES Members(member_id)
);
""")

# Data generation functions
def generate_authors(num=50):
    authors = []
    for _ in range(num):
        name = fake.name()
        birth_year = random.randint(1900, 1995)
        nationality = fake.country()
        authors.append((name, birth_year, nationality))
    return authors

def generate_books(num=200):
    books = []
    for _ in range(num):
        title = fake.sentence(nb_words=4)
        author_id = random.randint(1, 50)  # Assuming 50 authors
        publication_year = random.randint(1950, 2023)
        genre = random.choice(['Fiction', 'Non-fiction', 'Science', 'History', 'Fantasy'])
        price = round(random.uniform(5, 100), 2)
        availability_status = random.choice(['available', 'checked out', 'reserved'])
        books.append((title, author_id, publication_year, genre, price, availability_status))
    return books

def generate_members(num=100):
    members = []
    for _ in range(num):
        name = fake.name()
        membership_type = random.choice(['Basic', 'Premium'])
        joining_date = fake.date_between(start_date='-5y', end_date='today')
        total_books_borrowed = random.randint(0, 50)
        members.append((name, membership_type, joining_date, total_books_borrowed))
    return members

def generate_loans(num=1200):
    loans = []
    for _ in range(num):
        book_id = random.randint(1, 200)  # Assuming 200 books
        member_id = random.randint(1, 100)  # Assuming 100 members
        borrow_date = fake.date_between(start_date='-2y', end_date='today')
        return_date = borrow_date + timedelta(days=random.randint(1, 30)) if random.random() > 0.5 else None
        loan_status = 'returned' if return_date else 'overdue'
        loans.append((book_id, member_id, borrow_date, return_date, loan_status))
    return loans

# Insert data into tables
cursor.executemany("""
INSERT INTO Authors (name, birth_year, nationality) VALUES (?, ?, ?)
""", generate_authors())

cursor.executemany("""
INSERT INTO Books (title, author_id, publication_year, genre, price, availability_status)
VALUES (?, ?, ?, ?, ?, ?)
""", generate_books())

cursor.executemany("""
INSERT INTO Members (name, membership_type, joining_date, total_books_borrowed)
VALUES (?, ?, ?, ?)
""", generate_members())

cursor.executemany("""
INSERT INTO Loans (book_id, member_id, borrow_date, return_date, loan_status)
VALUES (?, ?, ?, ?, ?)
""", generate_loans())

# Commit and close
conn.commit()
conn.close()

print("Database created and populated successfully!")


Database created and populated successfully!


In [7]:
import os
os.getcwd()

'C:\\Users\\anton'